In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Problema:

Crie um agente capaz de encontrar o menor caminho entre duas cidades, com
mapa definido como segue. O agente deve receber como entradas o id da cidade
origem, id da cidade destino e nome do arquivo de dados. Usando este agente
encontre o menor caminho entre as cidades Alice Springs (id 5) e Yulara da
Australia(id 219) do arquivo australia.csv, explicite o caminho (a lista das cidades)
da solução e também a distância do início ao fim.

O arquivo australia.csv (disponível no site
da disciplina) tem os seguintes campos: ID da cidade,nome da cidade, coordenada x, coordenada y, estado e população. A distância em linha reta entre as cidades pode
ser calculada a partir das coordenadas cartesianas (x,y) disponibilizadas no arquivo
Australia.csv. Uma cidade com ID x se conecta com as cidades x+2 (se existir) e x-1, se x>1 e x é par. Se X é ímpar e x>2, esta cidade x se conecta com as cidades x-2 e
x+1(se existir cidade com ID x+1). Caso as cidades existam distância pela estrada é
10% maior que a distância em linha reta.

In [9]:
import numpy as np
import pandas as pd
import math
import networkx as nx
import igraph

In [10]:
def dist_cities_line(city1, city2):
    lat_x1 = float(data[data['id'] == city1]['lat'])
    lng_x1 = float(data[data['id'] == city1]['lng'])
    
    lat_x2 = float(data[data['id'] == city2]['lat'])
    lng_x2 = float(data[data['id'] == city2]['lng'])
    
    return math.sqrt((lat_x1 - lat_x2)**2 + (lng_x1 - lng_x2)**2)

def create_graph(data):
    road_factor = 1.1
    
    G = nx.Graph()
    data.apply(lambda x: G.add_node(x['id'], city= x['city']), axis=1)
    
    for city_id in list(G.nodes):
        if city_id > 1 and city_id%2 == 0:
            # Connects with x-1
            G.add_edge(city_id, city_id - 1, distance=road_factor*dist_cities_line(city_id, city_id - 1)) 
            # If exists, connects with x+2
            if city_id < len(data['city']) - 1:
                G.add_edge(city_id, city_id + 2, distance=road_factor*dist_cities_line(city_id, city_id + 2))       
        elif city_id%2 != 0 and city_id > 2:
            # Connects with x-2
            G.add_edge(city_id, city_id - 2, distance=road_factor*dist_cities_line(city_id, city_id - 2))
            # If exists, connects with x+1
            if city_id < len(data['city']):
                G.add_edge(city_id, city_id + 1, distance=road_factor*dist_cities_line(city_id, city_id + 1))
            
    return G

def get_road_distance(origin, destiny, G):
    try:
        distance = G[origin][destiny]["distance"]
    except:
        if origin == destiny:
            distance = 0
        else:
            distance = np.inf
    return distance
    
def find_route(origin, destiny, G):
    # List of all cities in the graph
    cities = list(G.nodes)
    # Calculated cost to go to the city from the origin, aiming the destiny
    arrival_cost = np.inf * np.ones(len(G.nodes))
    arrival_cost[origin-1] = 0
    # List of explored cities, just to avoid percurring the previous list complete
    # every time when is required to identify the explored cities 
    explored = [origin]
    # To reconstruct the route
    predecessor = np.zeros(len(cities))
    
  
    # Atributes the "False" value to "routed", and checks for the trivial case
    routed = False
    if(origin == destiny):
        routed = True 
    
    # Repeats while the route to destiny is not complete
    while (not routed):
        # A new city to explore
        explore = {}
        # Smallest cost to explore a new city until now
        smallest_exp_cost = np.inf
        # Calculate costs in the neighborhood of the explored cities
        for city in explored:
            
            # Just to make the algorithmru faster in this case
            #rang = range(max(1, city-3), min(len(cities)+1, city+3))
            
            #for neighbor in rang:
            for neighbor in G.neighbors(city):
                # Checks only the not explored cities
                if arrival_cost[neighbor-1] == np.inf:
                    # Calculates the expected cost function using the  straight line distance as heuristic                  
                    exp_cost = arrival_cost[city-1] + get_road_distance(city, neighbor, G) + dist_cities_line(neighbor, destiny)
                    # In the case the cost_function is smaller than the smallest until now,
                    # the first should take the latter's place 
                    # print("small:", smallest_exp_cost)
                    if exp_cost < smallest_exp_cost:                        
                        explore = {"city": neighbor, "expCost": exp_cost, "predecessor": city}
                        smallest_exp_cost = exp_cost
        
        # As the frontiers exploration ended, adds the new explored city
        explored.append(explore["city"])
        predecessor[explore["city"] - 1] = explore["predecessor"]
        arrival_cost[explore["city"] - 1] = smallest_exp_cost - dist_cities_line(explore["city"], destiny)       
        
        
        # If the algorithm found the destiny, then the route is done
        if(explore["city"] == destiny):
            routed = True      

    # Reconstructs the route
    start = destiny
    route = [destiny]
    while(start != origin):
        start = int(predecessor[start-1] )
        route.insert(0, start)
    
    return route, arrival_cost[destiny-1]


In [11]:
origin = 5 #Alice Sprnigs
destiny = 219 # Yulara
data = pd.read_csv('australia.csv')
G = create_graph(data)

In [12]:
final_answer = find_route(origin, destiny, G)

In [13]:
path_check = nx.dijkstra_path(G, source=origin, target=destiny, weight="distance")
distancy_check = 0
for i in range(0, len(path_check)-1):
    distancy_check = distancy_check + G[path_check[i]][path_check[i+1]]["distance"]

In [14]:
final_answer[0] == path_check

True

In [15]:
final_answer[1] == distancy_check

True